<img src="img/Brilliant_logo.png" width="20%">

### John Mahoney's Teaching Demo

mohnjahoney@gmail.com - mohnjahoney@github.io

In [88]:
# Imports
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import bqplot as bq
import bqplot.pyplot as plt
import numpy as np
import pandas as pd
import copy
import os
from sklearn.linear_model import LinearRegression
from IPython.display import display, HTML

# Simpson's Paradox

<hr style="height:6px">

Imagine you went to the grocery store and put 3 apples and 4 oranges in your shopping bag.
When you got to the register, you looked in the bag and found only 6 pieces of fruit - that would be strange, right?
Well, what if you looked inside and found **negative 6** pieces of fruit!?

That's a bit like how it feels to encounter **Simpson's Paradox** - our topic for this lesson.

<div class="alert alert-info" role="alert">
    <h2> Simpson's paradox in a nutshell: </h2>
    <p style="font-size:1.5em"> Combining subsets of data can result in counter-intuitive reversals of trends. </p>
    
<!--        <p class="lead"> Each of two data subsets displays the same trend, yet the whole dataset displays the opposite trend. </p> -->
</div>

In this lesson, we'll explore three curious scenarios, each of which creates a version of this paradox.

Our goals in this lesson are to:
- notice how our intuition may be wrong even in a simple situation,
- learn why our intuition fails, so that these examples seem less paradoxical,
- become aware of the different flavors of this paradox,
- understand how to act when faced with this paradox in real life.

Ready? Let's dig in!.

# EXAMPLE 1: The Shape of Fish.

<hr style="height:6px">

Here we see fish of many different sizes.
We can divide them into a red group and a blue group.
Notice that the red fish are generally "tall" while the blue fish are generally "long".

In [89]:
# Create fish data
fish = pd.DataFrame({'color':['red', 'red', 'red', 'blue', 'blue', 'blue'], 
                     'length':[0.5, 1.0, 1.5, 2.0, 3.0, 4.0], 
                     'height':[2.5, 3.5, 4.5, 1.0, 1.5, 2.0]})

# We don't want the image xspan to be as big as the length coordinate - otherwise the fish will overlap too much.
x_scale = 0.5
y_scale = 0.5

fish['xspan'] = x_scale * fish['length']
fish['yspan'] = y_scale * fish['height']

# We want the fish to start out somewhere "random".
fish['init_x'] = [4.2, 1.5, 3.9, 0.5, 3.1, 1.3]
fish['init_y'] = [0.6, 3.8, 4.4, 4.5, 2.3, 1]

#fish

In [90]:
def show_and_scale_fish(color, xs, ys):
    
    image_pathA = os.path.abspath('./img/fish_red.png')
    image_pathB = os.path.abspath('./img/fish_blue.png')
    
    if color == 'red':
        im = plt.imshow(image=image_pathA, format='filename')
    elif color == 'blue':
        im = plt.imshow(image=image_pathB, format='filename')
    
    # Scale
    im.x = xs
    im.y = ys
    
    return im

def show_and_scale_fish_from_df(row, init=False):
    
    color = row['color']
    
    if init:
        x = row['init_x']
        y = row['init_y']
    else:
        x = row['length']
        y = row['height']

    # NOTE: The extent of the image does not depend on whether it is in "initial" or "normal" position.
    dx = row['xspan']
    dy = row['yspan']
    
    xs = [x - dx/2, x + dx/2]
    ys = [y - dy/2, y + dy/2]
    
    im = show_and_scale_fish(color, xs, ys)
    return im

In [91]:
# Make initial "ocean" plot.

fig_ocean = plt.figure(min_aspect_ratio=1.0, max_aspect_ratio=1.01, 
                       fig_margin={'top':20, 'bottom':20, 'left':20, 'right':20})

fig_ocean.layout.width = '50%'
fig_ocean.layout.height = '400px'
fig_ocean.layout.border = 'green solid 1px'

# Add fish images
for i in range(len(fish)):
    show_and_scale_fish_from_df(fish.loc[i], init=True)

ocean_xs = np.linspace(0, 5, 400)
ocean_ys = -0.4 * np.abs(np.sin(4*ocean_xs)) + 6.0

ocean_line = plt.plot(ocean_xs, ocean_ys)
    
plt.xlim(0, 6)
plt.ylim(0, 6)

fig_ocean.axes[0].visible = False
fig_ocean.axes[1].visible = False

box_ocean = widgets.Box([fig_ocean], 
                        layout=widgets.Layout(border='black solid 1px'))

box_ocean

Box(children=(Figure(axes=[Axis(scale=LinearScale(max=6.0, min=0.0), visible=False), Axis(orientation='vertica…

## VISUALIZE THE DATA

After measuring the dimensions of each fish, we can organize each group in a graph according to their length and height.

In the graphs below, we see strong trends.
First focus on the red fish; As the fish get longer, they also get taller.
Now focus on the blue fish: We see a similar trend - as the fish get longer, they get taller too.

Given two quantities $A$ and $B$, 
when $A$ and $B$ tend to increase together, we say that $A$ and $B$ are **positively correlated**.
When $A$ and $B$ tend to change in opposite directions, we say that $A$ and $B$ are **negatively correlated**.

We just noticed that length and height increase together for the red fish and for the blue fish.
This means that for each group, the length and height are positively correlated.

We can visualize these correlations by drawing a line through each group of fish.
(Try the regression button.)
The slope of each line tells us quantitatively how much the fish height will change for a given change in fish length.

$$\textrm{slope} = \frac{\Delta y}{\Delta x} = \frac{\textrm{difference in fish height}}{\text{difference in fish length}}$$

In [92]:
# Plot the fish in their correct positions on two plots - one red, one blue.

reg_xs = np.linspace(0, 5, 2).reshape(-1, 1)

# Red fish
figA = plt.figure(min_aspect_ratio=1.0, max_aspect_ratio=1.01, 
                       fig_margin={'top':20, 'bottom':50, 'left':50, 'right':20})

figA.layout.width = '100%'
figA.layout.height = '400px'
figA.layout.border = 'green solid 1px'

# Add fish images
for i in range(len(fish)):
    if fish.loc[i]['color'] == 'red':
        show_and_scale_fish_from_df(fish.loc[i], init=False)

scatterA = plt.scatter(x=fish[fish['color']=='red']['length'], y=fish[fish['color']=='red']['height'], 
                      colors=['Red'])

regA = LinearRegression().fit(fish[['length']][fish['color']=='red'],
                              fish[['height']][fish['color']=='red'])

reg_ysA = regA.predict(reg_xs)
line_regA = plt.plot(x=reg_xs, y=reg_ysA, colors=['Pink'], stroke_width=0)

plt.xlabel('length')
plt.ylabel('height')

plt.xlim(0, 6)
plt.ylim(0, 6)

# Blue fish

figB = plt.figure(min_aspect_ratio=1.0, max_aspect_ratio=1.01, 
                       fig_margin={'top':20, 'bottom':50, 'left':50, 'right':20})

figB.layout.width = '100%'
figB.layout.height = '400px'
figB.layout.border = 'green solid 1px'

# Add fish images
for i in range(len(fish)):
    if fish.loc[i]['color'] == 'blue':
        show_and_scale_fish_from_df(fish.loc[i], init=False)

scatterB = plt.scatter(x=fish[fish['color']=='blue']['length'], y=fish[fish['color']=='blue']['height'], 
              colors=['Blue'])

regB = LinearRegression().fit(fish[['length']][fish['color']=='blue'],
                              fish[['height']][fish['color']=='blue'])

reg_ysB = regB.predict(reg_xs)
line_regB = plt.plot(x=reg_xs, y=reg_ysB, colors=['lightblue'], stroke_width=0)

plt.xlabel('length')
plt.ylabel('height')

plt.xlim(0, 6)
plt.ylim(0, 6)

# Regression
regression_button = widgets.ToggleButton(value=False, description='Show regression lines')

def on_regression_button(change, line_regA, line_regB):
    
    if change['new'] == True:
        # Show regression
        line_regA.set_trait('stroke_width', 4)
        line_regB.set_trait('stroke_width', 4)
    else:
        # Don't show 
        line_regA.set_trait('stroke_width', 0)
        line_regB.set_trait('stroke_width', 0)
        
regression_button.observe(lambda change:on_regression_button(change, line_regA, line_regB), 'value')

# Container
box_red_blue_figs = widgets.HBox([figA, figB])
box_red_blue = widgets.VBox([box_red_blue_figs, regression_button])

box_red_blue.layout.width = '800px'
box_red_blue.layout.border = 'black solid 1px'

box_red_blue

## QUICK QUIZ:

<div class="alert alert-success" role="alert">
</div>

- Find the blue fish that is 4 feet long. How tall is it?
(answer: 2 feet)

- Imagine you find a red fish that is 2 feet long. How tall do you expect it to be?
(answer: 5.5 feet)

- Blue fish: When the length increases by two feet, the height (increases / decreases) by ___ feet.
(answer: increases, 2/3)

- Think! Consider the number of fishermen ($A$) at a pier and the number of fish ($B$) in the water below. Would you expect that $A$ and $B$ are positively or negatively correlated? Can you argue both sides?

<!-- <div class="alert alert-info" role="alert">
</div> -->

## ...HERE COMES THE PARADOX...

We've seen that, within the group of red fish, longer fish are taller.
The same is true for blue fish.

<p style="font-size:1.5em"> <it>Certainly</it> if we were to consider all fish (red and blue) together, we would continue to find that longer fish are taller. Right?... </p>

Let's put all of the fish in the same graph and see what happens! (Try the aggregation button)

In [93]:
# Aggregate the two groups of fish. Draw an aggregate regression line.

reg_xs = np.linspace(0, 5, 2).reshape(-1, 1)

# All fish
figAB = plt.figure(min_aspect_ratio=1.0, max_aspect_ratio=1.01, 
                       fig_margin={'top':20, 'bottom':40, 'left':40, 'right':20})

figAB.layout.width = '100%'
figAB.layout.height = '400px'
figAB.layout.border = 'green solid 1px'

# Add fish images
for i in range(len(fish)):
    show_and_scale_fish_from_df(fish.loc[i], init=False)

# dots
scatterA2 = plt.scatter(x=fish[fish['color']=='red']['length'], y=fish[fish['color']=='red']['height'], 
                      colors=['Red'])
scatterB2 = plt.scatter(x=fish[fish['color']=='blue']['length'], y=fish[fish['color']=='blue']['height'], 
                      colors=['Blue'])

# Regressions
regA = LinearRegression().fit(fish[['length']][fish['color']=='red'],
                              fish[['height']][fish['color']=='red'])

reg_ysA = regA.predict(reg_xs)
line_regA2 = plt.plot(x=reg_xs, y=reg_ysA, colors=['Pink'], stroke_width=4)

regB = LinearRegression().fit(fish[['length']][fish['color']=='blue'],
                              fish[['height']][fish['color']=='blue'])

reg_ysB = regB.predict(reg_xs)
line_regB2 = plt.plot(x=reg_xs, y=reg_ysB, colors=['lightblue'], stroke_width=4)

regAB = LinearRegression().fit(fish[['length']],
                              fish[['height']])

reg_ysAB = regAB.predict(reg_xs)
line_regAB2 = plt.plot(x=reg_xs, y=reg_ysAB, colors=['lightgreen'], stroke_width=0)

plt.xlim(0, 6)
plt.ylim(0, 6)


# Aggregate + regression
fish_agg_button = widgets.ToggleButton(value=False, description='Aggregate groups')

# TODO: This is an ugly hack, but it's OK for today.
def on_fish_agg_button(change):
    if change['new'] == True:
        line_regAB2.set_trait('stroke_width', 4)
        line_regA2.set_trait('stroke_width', 0)
        line_regB2.set_trait('stroke_width', 0)
        scatterA2.set_trait('colors', ['green'])
        scatterB2.set_trait('colors', ['green'])
        for i in range(6):
            figAB.marks[i].x = figAB.marks[i].x * 100
    else:
        line_regAB2.set_trait('stroke_width', 0)
        line_regA2.set_trait('stroke_width', 4)
        line_regB2.set_trait('stroke_width', 4)
        scatterA2.set_trait('colors', ['red'])
        scatterB2.set_trait('colors', ['blue'])
        for i in range(6):
            figAB.marks[i].x = figAB.marks[i].x / 100
            
fish_agg_button.observe(on_fish_agg_button, 'value')


# Container
box_red_AND_blue = widgets.VBox([figAB, fish_agg_button])

box_red_AND_blue.layout.width = '400px'
box_red_AND_blue.layout.border = 'black solid 1px'

box_red_AND_blue

Surprisingly, when we aggregate the subgroups, the trend is reversed!
The length and height are now **negatively correlated**.

Note: There is no straight line that we can draw through the set of all fish.
So we draw a line that is "the best fit".
We use the very standard **least-squares regression line** <you can find out more in this other lesson!>.

<div class="alert alert-info" role="alert">
    <h2> Take-home Message </h2>
    <p style="font-size:1.5em"> For each subgroup, longer fish are generally taller. However, when we look at all fish together, longer fish are generally shorter.
    <br>
    <br>
    Two positive trends can combine to create a negative trend.</p>
</div>

## INTERACTIVE CHALLENGE!

We just discussed an example with two subgroups (of fish).
Let's stretch your understanding a bit.
Can you create a Simpson's Paradox with **three** subgroups?
Move the shapes (circles, squares, and triangles) so that the correlation is **negative within each group**, yet the **overall trend is positive**.

In [94]:
# Play the game!

xmax = 10
ymax = 9

xs = np.array([1.0, 4.0, 6.0, 4, 5, 8, 3, 5, 9])
ys = np.array([8.0, 6.0, 9.0, 2, 1, 4, 1, 4, 3])

circle_inds = [0, 1, 2]
square_inds = [3, 4, 5]
triangle_inds = [6, 7, 8]

fig_challenge1 = plt.figure(fig_margin={'top':0, 'bottom':0, 'left':0, 'right':0})
# min_aspect_ratio=1.0, max_aspect_ratio=1.01, 

fig_challenge1.layout.width = '100%'
fig_challenge1.layout.height = '300px'
fig_challenge1.layout.border = 'green solid 1px'

reg0 = LinearRegression().fit(xs[circle_inds].reshape(-1, 1), ys[circle_inds])
reg1 = LinearRegression().fit(xs[square_inds].reshape(-1, 1), ys[square_inds])
reg2 = LinearRegression().fit(xs[triangle_inds].reshape(-1, 1), ys[triangle_inds])
reg012 = LinearRegression().fit(xs.reshape(-1, 1), ys)

reg_xs = np.linspace(0, 10, 2)

reg_ys0 = reg0.predict(reg_xs.reshape(-1, 1))
reg_ys1 = reg1.predict(reg_xs.reshape(-1, 1))
reg_ys2 = reg2.predict(reg_xs.reshape(-1, 1))
reg_ys012 = reg012.predict(reg_xs.reshape(-1, 1))

line_reg0 = plt.plot(reg_xs, reg_ys0, colors=['red'], stroke_width=3)
line_reg1 = plt.plot(reg_xs, reg_ys1, colors=['blue'], stroke_width=3)
line_reg2 = plt.plot(reg_xs, reg_ys2, colors=['green'], stroke_width=3)
line_reg012 = plt.plot(reg_xs, reg_ys012, colors=['black'], stroke_width=6, line_style='dashed')

scatter0 = plt.scatter(xs[circle_inds], ys[circle_inds], default_size=600, colors=['white'], 
                       stroke='red', stroke_width=4, marker='circle', enable_move=True)
scatter1 = plt.scatter(xs[square_inds], ys[square_inds], default_size=600, colors=['white'], 
                       stroke='blue', stroke_width=4, marker='square', enable_move=True)
scatter2 = plt.scatter(xs[triangle_inds], ys[triangle_inds], default_size=600, colors=['white'], 
                       stroke='green', stroke_width=4, marker='triangle-up', enable_move=True)

# status_text = plt.label(["Not Yet"], x=[5], y=[11], 
#                             align='middle', font_weight='bold', default_size=24, colors=['Black'])

htmltext = "blah blah"
haha = widgets.HTML(value = f"<b><font color='red'>{htmltext}</b>", 
                          layout=widgets.Layout(border='white solid 1px'))

def randomize_positions(button, scatter0, scatter1, scatter2):
#    print("wer")
    scatter0.x = np.random.uniform(0, xmax, 3)
    scatter1.x = np.random.uniform(0, xmax, 3)
    scatter2.x = np.random.uniform(0, xmax, 3)
    scatter0.y = np.random.uniform(0, ymax, 3)
    scatter1.y = np.random.uniform(0, ymax, 3)
    scatter2.y = np.random.uniform(0, ymax, 3)
    
button_randomize = widgets.Button(
    description='Randomize positions',
    disabled=False,
    button_style='success', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Description')
    
button_randomize.on_click(lambda button: randomize_positions(button, scatter0, scatter1, scatter2))

def is_simpson(reg0, reg1, reg2, reg012, haha):
    conds = [(reg0.coef_ < 0), (reg1.coef_ < 0), (reg2.coef_ < 0), (reg012.coef_ > 0)]
    if conds[0]:
        line_reg0.set_trait('opacities', [1])
    else:
        line_reg0.set_trait('opacities', [0.2])
        
    if conds[1]:
        line_reg1.set_trait('opacities', [1])
    else:
        line_reg1.set_trait('opacities', [0.2])
        
    if conds[2]:
        line_reg2.set_trait('opacities', [1])
    else:
        line_reg2.set_trait('opacities', [0.2])
        
    if conds[3]:
        line_reg012.set_trait('opacities', [1])
    else:
        line_reg012.set_trait('opacities', [0.2])
      
    if sum(conds[:-1]) == 0:
        #status_text.text = ["Move the shapes"]
        status_text = "Move the shapes"
    elif sum(conds[:-1]) == 1:
        #status_text.text = ["You got one :)"]
        status_text = "You got one :)"
    elif sum(conds[:-1]) == 2:
        #status_text.text = ["You got two!!"]
        status_text = "You got two!!"
    elif sum(conds[:-1]) == 3 and conds[-1] == 0:
        #status_text.text = ["Now for the total!"]
        status_text = "Now for the total!"
    elif sum(conds) == 4:
        #status_text.text = ["You designed a Simpson's Paradox!"]
        status_text = "You designed a Simpson's Paradox!"
    else:
        print('ELSE+++++++++++++++++++++++')

    haha.value = "<b><font color='black'><font size='5em'>" + "{}".format(status_text) + "</b>"
    
def update_reg_line(change, scatter0, scatter1, scatter2, haha):
    reg0 = LinearRegression().fit(scatter0.x.reshape(-1, 1), scatter0.y.reshape(-1, 1))
    reg1 = LinearRegression().fit(scatter1.x.reshape(-1, 1), scatter1.y.reshape(-1, 1))
    reg2 = LinearRegression().fit(scatter2.x.reshape(-1, 1), scatter2.y.reshape(-1, 1))
    
    newxs = np.concatenate([scatter0.x, scatter1.x, scatter2.x])
    newys = np.concatenate([scatter0.y, scatter1.y, scatter2.y])
    reg012 = LinearRegression().fit(newxs.reshape(-1, 1), newys.reshape(-1, 1))
    
    reg_ys0 = reg0.predict(reg_xs.reshape(-1, 1))
    reg_ys1 = reg1.predict(reg_xs.reshape(-1, 1))
    reg_ys2 = reg2.predict(reg_xs.reshape(-1, 1))
    reg_ys012 = reg012.predict(reg_xs.reshape(-1, 1))

    line_reg0.y = reg_ys0
    line_reg1.y = reg_ys1
    line_reg2.y = reg_ys2
    line_reg012.y = reg_ys012
    
    is_simpson(reg0, reg1, reg2, reg012, haha)
    
scatter0.observe(lambda change: update_reg_line(change, scatter0, scatter1, scatter2, haha), names=['x','y'])
scatter1.observe(lambda change: update_reg_line(change, scatter0, scatter1, scatter2, haha), names=['x','y'])
scatter2.observe(lambda change: update_reg_line(change, scatter0, scatter1, scatter2, haha), names=['x','y'])
# scatter1.observe(update_reg_line, names=['x','y'])
# scatter2.observe(update_reg_line, names=['x','y'])

is_simpson(reg0, reg1, reg2, reg012, haha)
    
plt.xlim(0, xmax)
plt.ylim(0, 11)

#fig_challenge1.layout = widgets.Layout(width='100%', border='white solid 1px', height='400px')

#fig_challenge1.marks = fig_challenge1.marks# + [status_text]
fig_challenge1.axes[0].set_trait('visible', False)
fig_challenge1.axes[1].set_trait('visible', False)

box_challenge1 = widgets.VBox([haha, fig_challenge1])

box_challenge1.layout.border = 'red 1px solid'
box_challenge1.layout.width = '500px'
box_challenge1.layout.align_items = 'center'

# layout=widgets.Layout(border='red 1px solid', width='50%', 
#                                                     align_items='center', grid_gap='0px'))

box_challenge1

In [95]:
%%HTML
<style>
td {
  font-size: 24px
}
</style>

In [96]:
# Create batting dataframe

# Set up multi index dataframe for batting stats
years = [2018, 2019]
batting_details = ['hits', 'at-bats', 'BA']

mindex = pd.MultiIndex.from_product([years, batting_details],
                           names=['year', 'stats'])

# This will set dtypes as int. Batting ave will upcast to float later.
bat_df = pd.DataFrame(np.zeros(shape=(2, 6), dtype='int'), index=['Jackie', 'Arlo'], columns=mindex)

########
# Enter the main data: 'hits' and 'at bats'
bat_df.loc['Jackie'] = [1, 20, 0.0, 24, 80, 0.0]
bat_df.loc['Arlo'] = [12, 80, 0.0, 10, 20, 0.0]
########

# TODO: This could be done better.
bat_df.loc[:, ('Total', 'hits')] = bat_df.loc[:, (2018, 'hits')] + bat_df.loc[:, (2019, 'hits')]
bat_df.loc[:, ('Total', 'at-bats')] = bat_df.loc[:, (2018, 'at-bats')] + bat_df.loc[:, (2019, 'at-bats')]

def compute_BA(df):
    for year in df.columns.get_level_values('year').unique():
        df.loc[:, (year,'BA')] = df[year]['hits'] / df[year]['at-bats']
    return df

bat_df = compute_BA(bat_df)

# EXAMPLE 2: Batting Averages (BA)

<hr style="height:6px">

Jackie and Arlo have been playing baseball for the past two years.
Baseball fans argue about who is the better player - in particular, who has the better "batting average" (BA).

The batting average is defined as the ratio of the number of **hits** to the number of **at-bats**.

$$\textrm{BA} = \frac{\textrm{hits}}{\textrm{at-bats}}$$

For example, the table shows that in 2018 Jackie had only 1 hit in 20 at-bats.
Therefore his batting average for 2018 is:

$$\textrm{BA} = \frac{1}{20} = 0.05$$

In [97]:
# Show stats

seasons_out = widgets.Output()
overall_out = widgets.Output()

with seasons_out:
    display(bat_df[bat_df.columns[0:6]])
    
button_overall_stats = widgets.ToggleButton(description='Show overall', value=False)

def on_button_overall_stats(change):
    with overall_out:
        if change['new'] == True:
            display(bat_df[bat_df.columns[6:]])
        else:
            overall_out.clear_output()
        
button_overall_stats.observe(on_button_overall_stats, 'value')

stats_box = widgets.HBox([seasons_out, button_overall_stats, overall_out])
stats_box.layout.align_items = 'center'

stats_box

Some fans say "Arlo is definitely the better player! He had a better batting average in each of the two seasons.
(Look at the table and confirm that this is true.)

Other fans say, "Yeah, but what about the *overall batting average*?"
(Take a minute and compute each player's overall BA. This is the total number of hits divided by the total number of at-bats.)

Check your answer with the "show overall" button.

Hopefully you found that while Arlo has a better BA in each season, Jackie has the better total BA. 
How can this be?

We have another great example of Simpson's paradox!
Let's try to gain some intuition by graphing the data.

In [98]:
# slice(None) important for slicing a multi index
hits = bat_df.loc[:, (slice(None), 'hits')].to_numpy()
atbats = bat_df.loc[:, (slice(None), 'at-bats')].to_numpy()

# We want to plot the cumulative.
# -1 because we don't want the Total
hitscum = np.cumsum(hits[:, :-1], axis=1)
atbatscum = np.cumsum(atbats[:, :-1], axis=1)

# Prepend 0s for plotting
hitscum = np.hstack((np.zeros(shape=(2,1)), hitscum))
atbatscum = np.hstack((np.zeros(shape=(2,1)), atbatscum))

#print(hitscum)
#print(atbatscum)

In [108]:
# Make batting average interactive!

# Color choices
jackie_colors = ['red', 'red']
arlo_colors = ['blue', 'blue']

fig_batting = plt.figure(animation_duration=1000, 
                         min_aspect_ratio=1.5, max_aspect_ratio=1.51, 
                         fig_margin={'top':20, 'bottom':50, 'left':50, 'right':20})
fig_batting.layout.width = '100%'
fig_batting.layout.height = '400px'
fig_batting.layout.border = 'green solid 1px'

jackie_hits_cum = hitscum[0, :]
jackie_atbats_cum = atbatscum[0, :]

arlo_hits_cum = hitscum[1, :]
arlo_atbats_cum = atbatscum[1, :]

jackie_line_overall = plt.plot(x=jackie_atbats_cum[[0, -1]], y=jackie_hits_cum[[0, -1]], colors=['lightblue'], 
                               stroke_width=2, visible=False)
arlo_line_overall = plt.plot(x=arlo_atbats_cum[[0, -1]], y=arlo_hits_cum[[0, -1]], colors=['pink'], 
                             stroke_width=2, visible=False)

jackie_lines = []
arlo_lines = []
for ind in range(2):
    jackie_lines.append(plt.plot(x=atbatscum[0, [ind, ind+1]], y=hitscum[0, [ind, ind+1]], colors=[jackie_colors[ind]], 
                       stroke_width=5, line_style='solid'))
    arlo_lines.append(plt.plot(x=atbatscum[1, [ind, ind+1]], y=hitscum[1, [ind, ind+1]], colors=[arlo_colors[ind]], 
                       stroke_width=5, line_style='solid'))

jackie_lines[1].set_trait('line_style', 'dashed')
arlo_lines[1].set_trait('line_style', 'dashed')
jackie_lines[0].set_trait('labels', [''])
arlo_lines[0].set_trait('labels', [''])
jackie_lines[1].set_trait('labels', ['Jackie'])
arlo_lines[1].set_trait('labels', ['Arlo'])

vert_lines = []
for ind in [0, 1]:
    vert_lines.append(plt.plot(x=2*[atbatscum[ind, 1]], y=[0, 25], colors=['black'], line_style='solid', 
                       stroke_width=1))

jackie_text = plt.label(["Jackie"], x=[35], y=[20], align='middle', font_weight='bold', default_size=24, 
                      colors=['Red'])
arlo_text = plt.label(["Arlo"], x=[65], y=[20], align='middle', font_weight='bold', default_size=24, 
                      colors=['Blue'])


plt.xlabel("at-bats")
plt.ylabel("hits")

button_bat1 = widgets.ToggleButtons(
    value='season',
    options=['season', 'at-bats'],
    description='Sort by:',
    disabled=False,
    button_style='success', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Description',
    icon='check')

button_overall = widgets.ToggleButton(
    value=False,
    description='Show overall',
    button_style='success', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Description')
   
def switch_vals(arrA, arrB):
    temp = copy.copy(arrA)
    arrA = arrB
    arrB = temp
    return arrA, arrB

def on_button_bat1(change):
    # TODO: This function is rather terrible, but abstracting seems like a big pain right now.
    # Switch Arlo lines
    dx0 = np.diff(arlo_lines[0].x)[0]
    dy0 = np.diff(arlo_lines[0].y)[0]
    
    dx1 = np.diff(arlo_lines[1].x)[0]
    dy1 = np.diff(arlo_lines[1].y)[0]
    
    if change['new'] == 'at-bats':
        arlo_lines[1].x = [0, dx1]
        arlo_lines[1].y = [0, dy1]

        arlo_lines[0].x = [dx1, dx0 + dx1]
        arlo_lines[0].y = [dy1, dy0 + dy1]
        
    elif change['new'] == 'season':
        arlo_lines[0].x = [0, dx0]
        arlo_lines[0].y = [0, dy0]

        arlo_lines[1].x = [dx0, dx0 + dx1]
        arlo_lines[1].y = [dy0, dy0 + dy1]
    else:
        print(change)
    
    # Move second vert line
    if change['new'] == 'at-bats':
        vert_lines[1].x = x=2*[atbatscum[0, 1]]
    elif change['new'] == 'season':
        vert_lines[1].x = x=2*[atbatscum[1, 1]]
    else:
        print(change)
    
def on_button_overall(change):
    if change['new'] == True:
        jackie_line_overall.visible = True
        arlo_line_overall.visible = True
    else:
        jackie_line_overall.visible = False
        arlo_line_overall.visible = False

button_bat1.observe(on_button_bat1, names='value')

button_overall.observe(on_button_overall, names='value')

button_box = widgets.VBox([button_bat1, button_overall])
#button_box.layout.align_items = 'center'
button_box.layout.justify_content = 'space-around'
#button_box.layout.justify_items = 'flex-end'
button_box.layout.align_items = 'center'
#button_box.layout.align_content = 'space-around'
#button_box.layout.grid_gap = '20px 20px'
button_box.layout.height = '200px'
button_box.layout.border = 'blue solid 1px'

box_bat = widgets.HBox([fig_batting, button_box])
box_bat.layout.border = 'red solid 1px'
box_bat.layout.width = '800px'
box_bat.layout.height = '100%'
box_bat.layout.align_items = 'center'

box_bat

Here, we show hits vs at bats.
The solid lines represent the first season and the dashed lines represent the second season.
For example, the solid red line tells us that Jackie had only 1 hit in 20 at-bats in 2018.

The slope of each line segment represents the batting average for that period.
> We can see from the graph then that Arlo has a better BA for each season.

We can also graph at the overall statistics.
(Try the "Show Overall" button.)
These light-colored lines represent the overall hits and at-bats for each player.
And so the slope of this line represents their overall BA.
> We can see that Jackie has a better overall BA.

How can Arlo be better for each season, but Jackie be better overall?

To answer this question let's compare the two players in a different way.
Instead of sorting by season, let’s sort by comparable numbers of at-bats.
(Press the “at-bats” button.)

From this "at-bats perspective", we see that while Arlo has a better BA in the first chunk (of 20 at-bats), Jackie has a better BA in the second chunk (of 80 at-bats).
So who ought to be the overall winner?
This seems like much more of a toss-up now.. we are intuitively comfortable with it being either player.

> By comparing "apples to apples" (comparable numbers of at-bats), the paradox is dissolved.

When we compare the two players by season, we see that Arlo has a better BA each year (greater slope).
Yet somehow Jackie still comes out the overall leader in BA.

Let's try reorganizing this data: instead of sorting by season, let's sort by comparable at-bats.
From this vantage, Arlo still leads in one group (20 at-bats) but Jackie leads in the other (80 at-bats).
Since the leadership is now mixed, it should be no surprise that either player might attain the higher overall BA.

<div class="alert alert-info" role="alert">
    <h2> Take Home Message </h2>
    <p style="font-size:1.5em"> Leading in each season is not enough to ensure you will lead overall.</p>
    <br>
    <p style="font-size:1.5em"> When we compare "apples to apples" (comparable number of at-bats), the paradox is explained.</p>
</div>

## QUICK QUIZ

<div class="alert alert-success" role="alert">
</div>

What if Jackie and Arlo *each* had 20 at-bats in 2018 and 80 at-bats in 2019. Is Simpson’s paradox possible?

Let's switch gears and do a little algebra:

Imagine Jackie has $a$ hits in 2018 and $b$ hits 2019.
Similarly, Arlo has $c$ and $d$ hits respectively.

Assume that Arlo leads in batting average for each season:

Which of the following are true? (Check all that apply)

- $a / 20 < c / 20$ (+)
- $b / 80 < d / 80$ (+)
- $a < c$ (+)
- $b < d$ (+)

Jackie's overall average is $(a + b) / 100$ and Arlo's overall average is $(c + d) / 100$.

Putting this all together, which of the following are true? (Check all that apply)

- a + b  < c + d (+)
- a + c  < b + d 

Finally, $(a + b) / 100 < (c + d) / 100$ - Arlo must the overall leader, and there can be no Simpson’s paradox!

<div class="alert alert-info" role="alert">
    <h2> Take Home Message </h2>
    <p style="font-size:1.5em"> If both players have the same number of at-bats in any given season, Simpson's paradox is not possible.</p>
    <p style="font-size:1.5em"> Simpson's Paradox depends on an imbalance between players in the number of at-bats.</p>
</div>

# Interactive Challenge!

Now consider two new baseball players, Ichiro and Jose.
Imagine they have played for three seasons.
Can you design a scenario where Ichiro has a better BA for each season, but Joe has the better overall BA?
As you move the dots, notice the changes in the leaderboard.

In [113]:
# TODO add axis labels
# Make interactive batting challenge

# TODO: Make a toggle or second challenge or just quiz question where the at-bats for each section is the same for both players.

# fig_interactive_batting = plt.figure(layout=widgets.Layout(width='auto', height='auto'), animation_duration=100, 
#                                      fig_margin={'top':20, 'bottom':50, 'left':50, 'right':20}, display_legend=True)

fig_interactive_batting = plt.figure(animation_duration=100, 
                         min_aspect_ratio=2.0, max_aspect_ratio=2.01, 
                         fig_margin={'top':20, 'bottom':50, 'left':50, 'right':20})

fig_interactive_batting.layout.width = '100%'
fig_interactive_batting.layout.height = '300px'
fig_interactive_batting.layout.border = 'green solid 1px'

ichiro_hits_cum = np.array([0, 10, 20, 30])
ichiro_atbats_cum = np.array([0, 20, 40, 60])

jose_hits_cum = np.array([0, 12, 24, 36])
jose_atbats_cum = np.array([0, 20, 40, 60])

ichiro_lineA = plt.plot(x=ichiro_atbats_cum[0:2], y=ichiro_hits_cum[0:2], colors=['blue'], stroke_width=5)
ichiro_lineB = plt.plot(x=ichiro_atbats_cum[1:3], y=ichiro_hits_cum[1:3], colors=['blue'], stroke_width=5, line_style='dashed')
ichiro_lineC = plt.plot(x=ichiro_atbats_cum[2:4], y=ichiro_hits_cum[2:4], colors=['blue'], stroke_width=5, line_style='dotted')
jose_lineA = plt.plot(x=jose_atbats_cum[0:2], y=jose_hits_cum[0:2], colors=['red'], stroke_width=5)
jose_lineB = plt.plot(x=jose_atbats_cum[1:3], y=jose_hits_cum[1:3], colors=['red'], stroke_width=5, line_style='dashed')
jose_lineC = plt.plot(x=jose_atbats_cum[2:4], y=jose_hits_cum[2:4], colors=['red'], stroke_width=5, line_style='dotted')

ichiro_line_overall = plt.plot(x=ichiro_atbats_cum[[0, -1]], y=ichiro_hits_cum[[0, -1]], colors=['lightblue'], stroke_width=3)
jose_line_overall = plt.plot(x=jose_atbats_cum[[0, -1]], y=jose_hits_cum[[0, -1]], colors=['pink'], stroke_width=3)

ichiro_scatter0 = plt.scatter(x=[ichiro_atbats_cum[0]], y=[ichiro_hits_cum[0]], colors=['blue'], default_size=200, enable_move=False)
jose_scatter0 = plt.scatter(x=[jose_atbats_cum[0]], y=[jose_hits_cum[0]], colors=['red'], default_size=200, enable_move=False)

ichiro_scatter = plt.scatter(x=ichiro_atbats_cum[1:], y=ichiro_hits_cum[1:], colors=['blue'], default_size=200, enable_move=True)
jose_scatter = plt.scatter(x=jose_atbats_cum[1:], y=jose_hits_cum[1:], colors=['red'], default_size=200, enable_move=True)


ichiro_text = plt.label(["Ichiro"], x=[10], y=[30], align='middle', font_weight='bold', default_size=24, 
                      colors=['Blue'])
jose_text = plt.label(["Jose"], x=[25], y=[30], align='middle', font_weight='bold', default_size=24, 
                      colors=['Red'])

# status_text_congratulations = plt.label([""], x=[30], y=[20], 
#                             align='middle', font_weight='bold', default_size=24, colors=['Black'])

plt.xlabel("at-bats")
plt.ylabel("hits")

htmltext = "blah blah"
htmlWidget = widgets.HTML(value = f"<b><font color='red'>{htmltext}</b>")

def on_ichiro_move(change):
    if change['name'] == 'x':
        newxs = change['new']
        allnewxs = np.insert(newxs, 0, ichiro_scatter0.x[0])
        
        if np.any(np.diff(allnewxs) < 0):
            # Reject the move
            ichiro_scatter.x = change['old']
            return
        
        ichiro_lineA.x = allnewxs[0:2]
        ichiro_lineB.x = allnewxs[1:3]
        ichiro_lineC.x = allnewxs[2:4]
        ichiro_line_overall.x = allnewxs[[0, -1]]
        
    if change['name'] == 'y':
        newys = change['new']
        allnewys = np.insert(newys, 0, ichiro_scatter0.y[0])
        
        if np.any(np.diff(allnewys) < 0):
            # Reject the move
            ichiro_scatter.y = change['old']
            return
        
        ichiro_lineA.y = allnewys[0:2]
        ichiro_lineB.y = allnewys[1:3]
        ichiro_lineC.y = allnewys[2:4]
        ichiro_line_overall.y = allnewys[[0, -1]]

    is_simpson_bat(ichiro_lineA, ichiro_lineB, ichiro_lineC, jose_lineA, jose_lineB, jose_lineC)
    
def on_jose_move(change):
    if change['name'] == 'x':
        newxs = change['new']
        allnewxs = np.insert(newxs, 0, jose_scatter0.x[0])

        if np.any(np.diff(allnewxs) < 0):
            # Reject the move
            jose_scatter.x = change['old']
            return
        
        jose_lineA.x = allnewxs[0:2]
        jose_lineB.x = allnewxs[1:3]
        jose_lineC.x = allnewxs[2:4]
        jose_line_overall.x = allnewxs[[0, -1]]
        
    if change['name'] == 'y':
        newys = change['new']
        allnewys = np.insert(newys, 0, jose_scatter0.y[0])
        
        if np.any(np.diff(allnewys) < 0):
            # Reject the move
            jose_scatter.y = change['old']
            return
        
        jose_lineA.y = allnewys[0:2]
        jose_lineB.y = allnewys[1:3]
        jose_lineC.y = allnewys[2:4]
        jose_line_overall.y = allnewys[[0, -1]]
    
    is_simpson_bat(ichiro_lineA, ichiro_lineB, ichiro_lineC, jose_lineA, jose_lineB, jose_lineC)

def is_slope_greater(lineA, lineB):
    slopeA = np.diff(lineA.y) / np.diff(lineA.x)
    slopeB = np.diff(lineB.y) / np.diff(lineB.x)
    if slopeA > slopeB:
        return True
    else:
        return False
    
def is_simpson_bat(ichiro_lineA, ichiro_lineB, ichiro_lineC, jose_lineA, jose_lineB, jose_lineC):
    
    conds = [is_slope_greater(ichiro_lineA, jose_lineA), 
             is_slope_greater(ichiro_lineB, jose_lineB), 
             is_slope_greater(ichiro_lineC, jose_lineC), 
             is_slope_greater(ichiro_line_overall, jose_line_overall)]
    
    names = ["<font color='blue'>Ichiro", "<font color='red'>Jose"]
    if conds[0]:
        nameA = names[0]
    else:
        nameA = names[1]
        
    if conds[1]:
        nameB = names[0]
    else:
        nameB = names[1]
        
    if conds[2]:
        nameC = names[0]
    else:
        nameC = names[1]
        
    if conds[3]:
        name_overall = names[0]
    else:
        name_overall = names[1]
        
    if conds[0] and conds[1] and conds[2] and not conds[3]:
        congratulations = 'Congratulations!'
    else:
        congratulations = ''

    htmlWidget.value =  "" + \
    """
    <table style='width:200px;font-size:2.0em'>
      <tr>
        <th>Year</th>
        <th>Leader</th>
      </tr>
      <br>
      <tr>
        <td>2020</td>
        <td>{}</td>
      </tr>
      <tr>
        <td>2021</td>
        <td>{}</td>
      </tr>
      <tr>
        <td>2022</td>
        <td>{}</td>
      </tr>
      <tr>
        <td>Overall</td>
        <td>{}</td>
      </tr>
    </table>
    <br>
    <p style="font-size:2em">{}</p>
    """.format(nameA, nameB, nameC, name_overall, congratulations)
        
ichiro_scatter.observe(on_ichiro_move, names=['x', 'y'])
jose_scatter.observe(on_jose_move, names=['x', 'y'])

is_simpson_bat(ichiro_lineA, ichiro_lineB, ichiro_lineC, jose_lineA, jose_lineB, jose_lineC)

box_interactive_batting = widgets.HBox([fig_interactive_batting, htmlWidget])
box_interactive_batting.layout.border = 'red solid 1px'
box_interactive_batting.layout.width = '800px'

box_interactive_batting

## QUICK QUIZ:

<div class="alert alert-info" role="alert">
</div>

BLAH

Maybe we've had enough of baseball..


# EXAMPLE 3: Red Triangles

<hr style="height:6px">

There are ten objects divided into two groups of five.
Each object has a color (red or blue) and a shape (cross or triangle).
Initially, we are presented with gray rectangles - both the color and shape are unknown to us.

**Your goal is to find the group (upper/lower) with the most red triangles.**

The catch is that we are allowed to reveal either the shape or color, but not both at once.

- First, use the reveal button to see the shapes - which group has more triangles?
- Then use the button to reveal colors - which group has more red objects?
- Now make your choice! Which group do you suspect has the most red triangles?
- Reveal the "shape AND color" to see if you guessed right.

In [14]:
def red_triangles_interactive():
    """
    This interactive displays a flavor of Simpson's paradox.
    
    The user is asked whether there are more red triangles on the top or bottom level.
    
    In this interactive, the user can reveal either the color or the shape but not both.
    Based on this information they must make a choice of level.
    After they have chosen, they may reveal both the color and shape.
    If they choose "rationally", they will find that their choice is wrong.
    
    It is based on the the example found in the Martin Gardner book.
    Sometimes this example is seen as being about a woman looking for a man that is kind AND rich.
    She examines the groups bald men and notbald men.
    The notbald men are more likely to be kind.
    The notbald men are also more likely to be rich.
    However, because of the (anti) correlation between these features, the bald men are more likely to be kind AND rich.
    """
    
    # Each object has a position x, y and a color.
    num = 10
    colors = ['red', 'blue']
    shapes = ['triangle-up', 'cross']

    # This `df_init` remains untouched
    df_init = pd.DataFrame({'x':np.arange(num) % 5, 'y': [0 for ind in range(num//2)] + [1 for ind in range(num//2)], 
                       'color':[0, 0, 1, 0, 1, 1, 1, 0, 0, 1], 
                       'shape':[0, 1, 0, 1, 0, 1, 1, 0, 0, 1]})

    df_init['color'] = df_init['color'].map(dict(zip(np.arange(num), colors)))
    df_init['shape'] = df_init['shape'].map(dict(zip(np.arange(num), shapes)))

    # This dataframe `df` will be modified as we go.
    df = copy.copy(df_init)

    def get_shape_dfs(df):
        dfA = df[df['shape']==shapes[0]]
        dfB = df[df['shape']==shapes[1]]
        return dfA, dfB

    def permute_objects_in_df(df):
        n = len(df)
        halfn = n//2
        inds0 = np.random.choice(np.arange(halfn), replace=False, size=halfn)
        inds1 = np.random.choice(np.arange(halfn), replace=False, size=halfn) + halfn
        allinds = np.append(inds0, inds1)

        permuted_xs = df['x'][allinds].to_numpy()
        permuted_ys = df['y'][allinds].to_numpy() # Note that because we are permuting within rows this does nothing.

        df['x'] = permuted_xs
        df['y'] = permuted_ys

    def sync_scatters_position_w_df(scatterA, scatterB, df):
        # This can sync the scatter plots with either the initial or the active df.
    #     print("in sync")
        dfA, dfB = get_shape_dfs(df)

        scatterA.set_trait('x', dfA['x'].to_numpy())
        scatterA.set_trait('y', dfA['y'].to_numpy())

        scatterB.set_trait('x', dfB['x'].to_numpy())
        scatterB.set_trait('y', dfB['y'].to_numpy())
        
    def sync_scatters_w_df(scatterA, scatterB, df):
        # This is intended as a hard reset

        dfA, dfB = get_shape_dfs(df)

        scatterA.set_trait('x', dfA['x'].to_numpy())
        scatterA.set_trait('y', dfA['y'].to_numpy())

        scatterB.set_trait('x', dfB['x'].to_numpy())
        scatterB.set_trait('y', dfB['y'].to_numpy())
        
        show_color(scatterA, scatterB, df)
        show_shape(scatterA, scatterB)
        
    def on_button_reset(scatterA, scatterB, df, button_reveal, button_choice):
        print("reset")
        sync_scatters_w_df(scatterA, scatterB, df)
        
        # FIX: This apparently does not remove the "color and shape" option and I don't know why not.
        button_reveal.options = ['nothing', 'color', 'shape']
        button_reveal.value = 'nothing'
        
        button_choice.value = None

    def permute_scatter_positions(scatterA, scatterB, df):
        # Permute but within each row
        # We have to operate on both shape types at once.
    #     print("permute scatter position")

        permute_objects_in_df(df)
        sync_scatters_position_w_df(scatterA, scatterB, df)

    def hide_color(scatterA, scatterB):
        scatterA.colors = ['gray']
        scatterB.colors = ['gray']

    def show_color(scatterA, scatterB, df):
        dfA, dfB = get_shape_dfs(df)
        scatterA.colors = list(dfA['color'])
        scatterB.colors = list(dfB['color'])

    def hide_shape(scatterA, scatterB):
        scatterA.set_trait('marker', 'square')
        scatterB.set_trait('marker', 'square')

    def show_shape(scatterA, scatterB):
        scatterA.set_trait('marker', 'triangle-up')
        scatterB.set_trait('marker', 'cross')

    def on_button_choice(change, button_reveal):
    #     print("button choice")
        curr_val = button_reveal.value
        button_reveal.set_trait('options', ('nothing', 'color', 'shape', 'color and shape'))
        button_reveal.value = curr_val

    def on_button_reveal(change, scatterA, scatterB, df):
        # Reveal the requested properties.
        # Shape AND color only accessible after a level has been chosen

        state = change['new']

        if state == 'nothing':
            # Hide color and shape.
            hide_color(scatterA, scatterB)
            hide_shape(scatterA, scatterB)
        elif state == 'color':
            # Hide shape and show color.
            show_color(scatterA, scatterB, df)
            hide_shape(scatterA, scatterB)
        elif state == 'shape':
            # Hide color and show shape.
            hide_color(scatterA, scatterB)
            show_shape(scatterA, scatterB)
        elif state == 'color and shape':
            show_color(scatterA, scatterB, df)
            show_shape(scatterA, scatterB)
        else:
            print("not implemented yet")

        # If we have made a decision, don't continue to permute.
        # We want the user to see how their guess corresponds to reality without any extra confusion.
        if button_choice.value is None:
            permute_scatter_positions(scatterA, scatterB, df)

    # Make a figure showing these objects

    fig = plt.figure(ax=[], layout={'height':'300px', 'width':'500px', 'border':'black solid 2px'}, animation_duration=1000)

    dfA, dfB = get_shape_dfs(df)

    scatterA = plt.scatter(dfA['x'], dfA['y'], colors=list(dfA['color']), marker=shapes[0], default_size=1000)
    scatterB = plt.scatter(dfB['x'], dfB['y'], colors=list(dfB['color']), marker=shapes[1], default_size=1000)

    div_line = plt.plot([-0.5, 4.5], [0.5, 0.5], colors=['black'])

    # Create the buttons
    button_choice = widgets.RadioButtons(description="Choose:", options=['upper', 'lower'], value=None, 
                                         layout={'width':'150px'})

    button_reveal = widgets.Dropdown(description="Reveal:", options=['nothing', 'color', 'shape'], value='nothing', 
                                    layout={'border':'white solid 1px', 'width':'225px'})

    button_permute_positions = widgets.Button(description='permute positions')
    button_reset = widgets.Button(description='reset')

    # I think on_click wants a `change` - use lambda to make a dummy variable.
    button_choice.observe(lambda change: on_button_choice(change, button_reveal), 'value')
    button_reveal.observe(lambda change: on_button_reveal(change, scatterA, scatterB, df), 'value')

    button_permute_positions.on_click(lambda change: permute_scatter_positions(scatterA, scatterB, df))
    button_reset.on_click(lambda change: on_button_reset(scatterA, scatterB, df_init, button_reveal, button_choice))

    fig.axes[0].visible = False
    fig.axes[1].visible = False

    hide_color(scatterA, scatterB)
    hide_shape(scatterA, scatterB)

    rightbox = widgets.VBox([button_reveal, button_reset],
                             layout={'border':'white solid 1px', 'height':'100px'})
    rightbox.layout.align_items = 'center'
    rightbox.layout.justify_content = 'space-around'
    
    box = widgets.AppLayout(left_sidebar=button_choice, center=fig, right_sidebar=rightbox, 
                            pane_widths=[1, 3, 1.5], 
                            layout=widgets.Layout(border='black solid 1px', 
                                                  width='auto',
                                                  align_items='center'),
                            grid_gap='0px')
    
    return box

In [15]:
red_triangles_box = red_triangles_interactive()
red_triangles_box

AppLayout(children=(RadioButtons(description='Choose:', layout=Layout(grid_area='left-sidebar', width='150px')…

TODO: If we set up the puzzle right, maybe we don't have to walk through these details as much?
Save for quiz?

## QUICK QUIZ:

Click the Shape button.

- Which group has more triangles?
- Which group has more red shapes?
- Which group has the most red triangles?

- How can we best summarize this situation?

While there are more triangles in the bottom group and there are more red shapes in the bottom group, there are more red triangles in the top group.

Which details lead to this situation?:
- "Red" and "Triangle" go together on top but not on bottom. (+)
- 



<div class="alert alert-info" role="alert">
    <h2> Take Home Message </h2>
    <p style="font-size:1.5em"> Just because the bottom group has more red shapes and more triangles, this doesn't mean that it has more red triangles.</p>
</div>

This is possible because of how shape and color are correlated.
On the top, redness is positively correlated with triangleness - they go together.
On the bottom, redness is negatively correlated with triangleness.

## Wrap up

<hr style="height:6px">

### Fish

In this example, the fish had three independent properties: length, width, and color.

We divided the fish into two subgroups based on one property - color.
We found a correlation between length and width within each color subgroup.
We also found that this correlation was reversed when we aggregated color subgroups.

### Batting averages

In this example, the data has four independent properties: player, season, hits, and at-bats.

We divided the data into two subgroups based on two properties - player and season.
We found that the ratio (correlation) between hits and at-bats favored the same player in each season.
We found that this ratio (correlation) was reversed when we aggregated by season.

### Red triangles

In this example, the data has three independent properties: level, shape, and color.

We divided the objects into subgroups based on one property - level.
We found a correlation between level and shape.
We found a correlation between level and color.
We found that when we create a composite property (shape AND color) that its correlation with level is reversed.


## Complicated example

Show one of the canonical examples, e.g. the effect of a drug in a table.

Lots of numbers.
Somehow, "adding up" small things results in a larger result than adding up large things.
This is Simpson's paradox.

You can check these numbers to confirm the story - a pain, but straightforward.

Clearly this is *important* (we want to know which drugs to take and we want our doctors to know).

- Show the number table
- Compute for them how simpsons paradox arises (they don't have to do any math, but they can believe it).
- Question 1: Choose the sentence that explains ..? the source of the paradox?
- Question 2: Which drug should you choose? A - #1, B - #2, C - either, D - depends on if you are a man or woman

TODO: can we decouple this example from gender?

## QUICK QUIZ: 

<div class="alert alert-success" role="alert">
</div>

Which of the following *best* characterizes Simpson's paradox?

- A dataset may display both a positive and negative trend at the same time.
- A trend changes sign upon addition of just a single datapoint.
- A trend changes sign when data are combined. (YES)

Which statement best summarizes the origin of Simpsons paradox?

Fish:
- Each data subset trends upward, but the two subsets themselves are arranged in a downward fashion. (+)
- Each subgroup falls along a straight line, but the aggregate does not.
- Whenever you combine subsets with similar trends, the aggregate trend will reverse.

Batting averages:
- Batting averages are *ratios*. To compute the overall BA, 

<!-- <div class="alert alert-info" role="alert">
</div> -->

### Brilliant Teaching Principles (https://brilliant.org/principles/):

<hr style="height:6px">

Highlight the ways in which this lesson attends to these principles.

1. Excites: The greatest challenges to education are disinterest and apathy.

Fish are kind of silly looking.
TODO: Maybe move in that direction for the shape example or at least for the story.

2. Cultivates curiosity: Questions and storytelling that cultivate natural curiosity are better than the threat of a test.

No threats here.
Incorporate a range of difficulty including some very beginner.
Also, the interactive "games" have no time limit - this deemphasizes success vs failure, certainly in a test sense.
    
3. Is active: Effective learning is active, not passive. Watching a video is not enough.

Questions interspersed maintain engagement.
Interactive games.

4. Is applicable: Use it or lose it: it is essential to apply what you're learning as you learn it.

Drug example is very applicable.
Making decisions (or even just understanding your doctor) in the face of statistics is a valuable skill.

5. Is community driven: A community that challenges and inspires you is invaluable.

Not sure how to incorporate this.

6. Doesn't discriminate: Your age, country, and gender don't determine what you are capable of learning.

I took care to choose examples that seemed accessible to a wide audience.
Fish are innocuous.
Simple shapes and colors are accessible across grades, languages, cultures.
Simpson examples often include gender - something I avoided for this reason.

7. Allows for failure: The best learners allow themselves to make many mistakes along their journey.

TODO: Offer more opportunity for mistakes (that can be then fixed).

8. Sparks questions: The culmination of a great education isn't knowing all the answers — it's knowing what to ask.

TODO: End with some good food for thought. Maybe one mathy one and one practical.


# Lessons

- Simpson's paradox has something to do with aggregation and things switching.

- Be careful when you add: are you adding **numbers** (stuff) or **ratios** (not stuff)?

Example: Baseball batting averages can switch when we aggregate. However, **hits** will not switch.

- There are limits:
        you can't combine (batting aves) {0.1, 0.2, 0.3} and overcome {0.6, 0.7, 0.8} by aggregating - there has to be some overlap.
        
- What if drug A is better than B for women and men, but B is better for people as a group? 

- People seem to be comfortable with the idea that if 2019 BA was x and 2020 was y, then the aggregate should be between x and y. (and this is true)
- People seem (fairly) comfortable with weighted averages. If there were many more at bats in 2019, then the aggregate should be closer to x than to y.
- If we have two batter and the at bat distribution is the same for 2019 and 2020, then there can be no SP.
- If the two batters have overlapping BAs, and different yearly at bats, and the "lower" one gets pulled up and the "higher" one gets pulled down, the result can be a flip.

- OK, but what should you *do*?

Take the example of males and females under the old and new education plans.
The new plan increases scores of both groups, but the overall average score is lower.
So what should you do next year? use the new plan or not?

## QUICK QUIZ

<div class="alert alert-success" role="alert">
</div>

- The slope of each line represents the ______ for that season. (batting average)

- We can see that Arlo has the better batting average for each season because:
Each blue line is higher than the corresponding red one.
Each blue line has a greater slope than the corresponding red one. (+)
The blue line sequence ends up with a greater final value than the red one.

- Something about how this process differs from adding fractions.

From the “at-bats” view, Arlo leads in the small chunk while Jackie leads in the large chunk.
Does this necessarily mean that Jackie must be the overall leader?
Can you modify Jackie’s 2019 hits so that he still leads over the 80 at-bats but is no longer the overall leader?

**DO THIS**
Imagine that each player had 100 at-bats in each season with the same batting averages reported above.
Fill in the table so that this is true.
Who has the greater overall BA? (answer: Arlo)

This tells us that knowing just the batting averages alone is not enough information.
We need to know the details - the hits and the at-bats.

**DO THIS**
If Arlo's worst BA is better than Jackie's best BA, can we have a Simpson's paradox situation?
(answer: no)
We can create a nice graphical proof of this.
How to incorporate?
A - Notice: here's a graphic that proves this fact.
B - Here's a graphic. Which fact does it prove?
C - Walk through the construction of the graphical proof.